In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
PATH = os.getcwd().replace("code", "")
Simpler_instance = PATH + "data\\Simpler_instance\\"

os.listdir(Simpler_instance)

['aircraft.csv',
 'aircraft_restrictions.csv',
 'aircraft_unavailabilities.csv',
 'airports.csv',
 'airport_closures.csv',
 'cabin_capacities.csv',
 'crew.csv',
 'crew_groups.csv',
 'crew_pairings.csv',
 'crew_rostering.csv',
 'crew_unavailabilities.csv',
 'flights.csv',
 'flight_disruptions.csv',
 'itineraries.csv',
 'maintenance.csv',
 'meal_costs.csv',
 'min_cabin_crew.csv',
 'min_con_pax.csv',
 'min_ground_time.csv',
 'min_pilots.csv',
 'routes.csv',
 'slots.csv',
 'slot_changes.csv',
 'visa_matrix.csv']

In [3]:
# obligatory
obligatory = [
    "aircraft.csv",
    # "aircraft_restrictions.csv",
    "aircraft_unavailabilities.csv",
    # "airports.csv",
    "airport_closures.csv",
    # "cabin_capacities.csv",
    "crew.csv",
    # "crew_groups.csv",
    "crew_pairings.csv",
    "crew_rostering.csv",
    "crew_unavailabilities.csv",
    "flights.csv",
    "flight_disruptions.csv",
    # "itineraries.csv",
    "maintenance.csv",
    # "meal_costs.csv",
    # "min_cabin_crew.csv",
    # "min_con_pax.csv",
    # "min_ground_time.csv",
    # "min_pilots.csv",
    "routes.csv",
    # "slots.csv",
    # "slot_changes.csv",
    # "visa_matrix.csv",
]

In [4]:
for file in obligatory:
    print(
        "File: {}\n{}\n{}".format(
            file,
            pd.read_csv(Simpler_instance + file),
            "-" * 50,
        )
    )

File: aircraft.csv
  registration model  family position  owned
0       A321#4  A321  Airbus      CDG      1
1      A320#10  A320  Airbus      TLS      1
--------------------------------------------------
File: aircraft_unavailabilities.csv
  registration             start            finish
0      A320#10  2006-01-07T18:20  2006-01-07T18:30
--------------------------------------------------
File: airport_closures.csv
  code             start            finish
0  TLS  2006-01-07T18:20  2006-01-07T18:25
--------------------------------------------------
File: crew.csv
          id position rank base acclimatization  duty_time  rest_time  \
0   JMOHXOTO      CDG   CP  CDG             CDG          0       1200   
1   KSSTFGJL      CDG   FO  CDG             CDG          0       1200   
2   GUCNZGXL      CDG   CS  CDG             CDG          0       1200   
3   IDKJZBQD      CDG   CS  CDG             CDG          0       1200   
4   BQSEGYAI      CDG   ST  CDG             CDG          0    

In [5]:
obligatory

['aircraft.csv',
 'aircraft_unavailabilities.csv',
 'airport_closures.csv',
 'crew.csv',
 'crew_pairings.csv',
 'crew_rostering.csv',
 'crew_unavailabilities.csv',
 'flights.csv',
 'flight_disruptions.csv',
 'maintenance.csv',
 'routes.csv']

In [6]:
flights = pd.read_csv(Simpler_instance + "flights.csv")
flight_disruptions = pd.read_csv(Simpler_instance + "flight_disruptions.csv")
aircraft = pd.read_csv(Simpler_instance + "aircraft.csv")
aircraft_unavailabilities = pd.read_csv(
    Simpler_instance + "aircraft_unavailabilities.csv"
)
airport_closures = pd.read_csv(Simpler_instance + "airport_closures.csv")
maintenance = pd.read_csv(Simpler_instance + "maintenance.csv")
routes = pd.read_csv(Simpler_instance + "routes.csv")

In [23]:
departures_board = pd.merge(
    pd.merge(
        pd.merge(
            pd.merge(
                pd.merge(flights, routes, how="left"),
                flight_disruptions,
                how="left",
            ).fillna(0),
            maintenance,
            how="left",
        ).assign(
            maintenance_unavailabilitie=lambda x: (
                (x.min_start >= x.start) & (x.min_start <= x.finish)
            )
            | ((x.max_finish >= x.start) & (x.max_finish <= x.finish))
        ),
        aircraft_unavailabilities.rename(
            columns={"start": "start_unv", "finish": "finish_unv"}
        ),
        left_on="aircraft",
        right_on="registration",
        how="left",
    ).assign(
        aircraft_unavailabilities=lambda x: (
            (x.start_unv >= x.start) & (x.start_unv <= x.finish)
        )
        | ((x.finish_unv >= x.start) & (x.finish_unv <= x.finish))
    ),
    airport_closures.rename(columns={"start": "start_cls", "finish": "finish_cls"}),
    left_on="from",
    right_on="code",
    how="left",
).assign(
    airport_closures=lambda x: ((x.start_cls >= x.start) & (x.start_cls <= x.finish))
    | ((x.finish_cls >= x.start) & (x.finish_cls <= x.finish))
)

cols = departures_board.loc[
    :,
    (
        departures_board.columns.str.contains("start")
        | departures_board.columns.str.contains("finish")
    ),
].columns
departures_board[cols] = departures_board[cols].apply(pd.to_datetime, errors="coerce")

departures_board

,leg_id,flight_id,date,aircraft,from,to,start,finish,crew_group_id,type,...,airports,maintenance_unavailabilitie,registration,start_unv,finish_unv,aircraft_unavailabilities,code,start_cls,finish_cls,airport_closures
0,70002970,2970,2006-01-07,A320#10,TLS,ORY,2006-01-07 06:40:00,2006-01-07 08:00:00,376980210,D,...,"TLS,ORY",False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False
1,70002975,2975,2006-01-07,A320#10,ORY,TLS,2006-01-07 08:50:00,2006-01-07 10:00:00,376980210,D,...,"TLS,ORY",False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,NaN,NaT,NaT,False
2,70002982,2982,2006-01-07,A320#10,TLS,ORY,2006-01-07 10:40:00,2006-01-07 12:00:00,376980210,D,...,"TLS,ORY",False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False
3,70002985,2985,2006-01-07,A320#10,ORY,TLS,2006-01-07 12:50:00,2006-01-07 14:00:00,376980210,D,...,"TLS,ORY",False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,NaN,NaT,NaT,False
4,70002992,2992,2006-01-07,A320#10,TLS,ORY,2006-01-07 14:40:00,2006-01-07 16:00:00,1878748607,D,...,"TLS,ORY",False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False
5,70002999,2999,2006-01-07,A320#10,ORY,TLS,2006-01-07 16:50:00,2006-01-07 18:00:00,1878748607,D,...,"TLS,ORY",False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,NaN,NaT,NaT,False
6,70003008,3008,2006-01-07,A320#10,TLS,ORY,2006-01-07 18:40:00,2006-01-07 20:00:00,1827753950,D,...,"TLS,ORY",True,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False
7,70004563,4563,2006-01-07,A321#4,CDG,NCE,2006-01-07 06:20:00,2006-01-07 07:55:00,623057530,D,...,NaN,False,NaN,NaT,NaT,False,NaN,NaT,NaT,False
8,70004564,4564,2006-01-07,A321#4,NCE,CDG,2006-01-07 09:05:00,2006-01-07 10:40:00,623057530,D,...,NaN,False,NaN,NaT,NaT,False,NaN,NaT,NaT,False
9,70004567,4567,2006-01-07,A321#4,CDG,NCE,2006-01-07 11:35:00,2006-01-07 13:10:00,623057530,D,...,NaN,False,NaN,NaT,NaT,False,NaN,NaT,NaT,False


In [85]:
# def loss_function(df: pd.DataFrame) -> float:
#     delta = df.delta
#     if df.maintenance_unavailabilitie:
#         temp = df.duration
#     return temp
def loss_function(
    delta: float,
    maintenance_unavailabilitie: bool,
    duration: float,
    aircraft_unavailabilities: bool,
    start_unv,
    finish_unv,
    airport_closures: bool,
    start_cls,
    finish_cls,
) -> float:
    val = delta
    if maintenance_unavailabilitie:
        val += duration
    if aircraft_unavailabilities:
        val += (finish_unv - start_unv).total_seconds() / 60.0
    if airport_closures:
        val += (finish_cls - start_cls).total_seconds() / 60.0

    return val

In [93]:
# departures_board.apply(lambda x: loss_function(x))

# https://stackoverflow.com/questions/47372274/apply-custom-function-over-multiple-columns-in-pandas
loss_function_vectorized = np.vectorize(loss_function)
departures_board["loss"] = loss_function_vectorized(
    delta=departures_board.delta,
    maintenance_unavailabilitie=departures_board.maintenance_unavailabilitie,
    duration=departures_board.duration,
    aircraft_unavailabilities=departures_board.aircraft_unavailabilities,
    start_unv=departures_board.start_unv,
    finish_unv=departures_board.finish_unv,
    airport_closures=departures_board.airport_closures,
    start_cls=departures_board.start_cls,
    finish_cls=departures_board.finish_cls,
)

print(sum(departures_board.loss))

departures_board

426.0


,leg_id,flight_id,date,aircraft,from,to,start,finish,crew_group_id,type,...,maintenance_unavailabilitie,registration,start_unv,finish_unv,aircraft_unavailabilities,code,start_cls,finish_cls,airport_closures,loss
0,70002970,2970,2006-01-07,A320#10,TLS,ORY,2006-01-07 06:40:00,2006-01-07 08:00:00,376980210,D,...,False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False,39.0
1,70002975,2975,2006-01-07,A320#10,ORY,TLS,2006-01-07 08:50:00,2006-01-07 10:00:00,376980210,D,...,False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,NaN,NaT,NaT,False,39.0
2,70002982,2982,2006-01-07,A320#10,TLS,ORY,2006-01-07 10:40:00,2006-01-07 12:00:00,376980210,D,...,False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False,48.0
3,70002985,2985,2006-01-07,A320#10,ORY,TLS,2006-01-07 12:50:00,2006-01-07 14:00:00,376980210,D,...,False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,NaN,NaT,NaT,False,0.0
4,70002992,2992,2006-01-07,A320#10,TLS,ORY,2006-01-07 14:40:00,2006-01-07 16:00:00,1878748607,D,...,False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False,0.0
5,70002999,2999,2006-01-07,A320#10,ORY,TLS,2006-01-07 16:50:00,2006-01-07 18:00:00,1878748607,D,...,False,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,NaN,NaT,NaT,False,0.0
6,70003008,3008,2006-01-07,A320#10,TLS,ORY,2006-01-07 18:40:00,2006-01-07 20:00:00,1827753950,D,...,True,A320#10,2006-01-07 18:20:00,2006-01-07 18:30:00,False,TLS,2006-01-07 18:20:00,2006-01-07 18:25:00,False,300.0
7,70004563,4563,2006-01-07,A321#4,CDG,NCE,2006-01-07 06:20:00,2006-01-07 07:55:00,623057530,D,...,False,NaN,NaT,NaT,False,NaN,NaT,NaT,False,0.0
8,70004564,4564,2006-01-07,A321#4,NCE,CDG,2006-01-07 09:05:00,2006-01-07 10:40:00,623057530,D,...,False,NaN,NaT,NaT,False,NaN,NaT,NaT,False,0.0
9,70004567,4567,2006-01-07,A321#4,CDG,NCE,2006-01-07 11:35:00,2006-01-07 13:10:00,623057530,D,...,False,NaN,NaT,NaT,False,NaN,NaT,NaT,False,0.0
